In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn import neighbors,svm,tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,AdaBoostRegressor,BaggingRegressor,ExtraTreesRegressor
from sklearn.linear_model import LinearRegression,Ridge,BayesianRidge, ElasticNet ,Lasso

from math import sqrt
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

#### Coordinate transformation

In [ ]:
# xy->XY
def xy2XY(x, y, geo):
    X = geo[0] + x * geo[1] + y * geo[2]
    Y = geo[3] + x * geo[4] + y * geo[5]
    return X, Y


def xy2XY_2(xy, geo):
    X = geo[0] + xy[0] * geo[1] + xy[1] * geo[2]
    Y = geo[3] + xy[0] * geo[4] + xy[1] * geo[5]
    return X, Y


# XY->xy
def XY2xy(X, Y, geo):
    a = np.array([[geo[1], geo[2]], [geo[4], geo[5]]])
    b = np.array([X - geo[0], Y - geo[3]])
    x, y = np.linalg.solve(a, b)
    return round(x), round(y)


### Set input and output

In [ ]:
# The number of pairs of images to be adjusted
IMG_pair_num = 7

#Input the grid size of the elevation error 
'''
excluding the extended parts on both sides
'''
Grid_Size = 5   # 5，9，17
MAX_Initial_Grid_Size=17
skip_step=int((MAX_Initial_Grid_Size-1)/(Grid_Size-1))

# Type of search grid
Herr_Grid_Method_type=["MEAN","MAE","RMSE"]
Herr_Grid_Method=Herr_Grid_Method_type[2]

# Read Initial search grid
Herr_Grid = np.load("dataset/PA2/Initial_Herr_Grid/"+Herr_Grid_Method+"_1.npy")

# Size of predict grid
Predict_Grid_Size = 101


In [ ]:
# Select predict model
model_line=LinearRegression()
model_rd=Ridge()
model_br=BayesianRidge()
model_en=ElasticNet()
model_ls=Lasso()

model_rf=RandomForestRegressor(max_depth= 5,min_samples_leaf=1,min_samples_split=6,n_estimators=151)
model_bg=BaggingRegressor(n_estimators=150)
model_et=ExtraTreesRegressor(max_depth= 5,min_samples_leaf=1,min_samples_split=4,n_estimators=38)
model_ada=AdaBoostRegressor(n_estimators=100,learning_rate=0.2)
model_gbdt=GradientBoostingRegressor(n_estimators=6,learning_rate=0.32)

# model_dic =[model_rf]
model_dic =[model_line,model_rd,model_br,model_en,model_ls,model_rf,model_bg,model_et,model_ada,model_gbdt]
model_name=['Line','Ridge','BayesianRidge','ElasticNet','Lasso','RandomForest','Bagging','ExtraTrees','AdaBoost','GradientBoosting']
Model_num=len(model_dic)


In [ ]:
# Output
# The coordinates and predicted values of the five smallest positions
Best_Initial_Position=np.zeros((IMG_pair_num, Model_num,5,3))

# Model fitting evaluation results
Model_evaluation=np.zeros((IMG_pair_num, Model_num,3))


### Generate a dataset

In [ ]:
# Generate the input coordinate grid
Normal_Step = 1 / (Grid_Size - 1) # Normalized grid step size

data_X = np.zeros((Grid_Size * Grid_Size, 2))
for i in range(0, Grid_Size * Grid_Size):
    data_X[i, 0] = (i // Grid_Size) * Normal_Step - 0.5
    data_X[i, 1] = (i % Grid_Size) * Normal_Step - 0.5


print(data_X)
print(data_X.shape)

[[-0.5  -0.5 ]
 [-0.5  -0.25]
 [-0.5   0.  ]
 [-0.5   0.25]
 [-0.5   0.5 ]
 [-0.25 -0.5 ]
 [-0.25 -0.25]
 [-0.25  0.  ]
 [-0.25  0.25]
 [-0.25  0.5 ]
 [ 0.   -0.5 ]
 [ 0.   -0.25]
 [ 0.    0.  ]
 [ 0.    0.25]
 [ 0.    0.5 ]
 [ 0.25 -0.5 ]
 [ 0.25 -0.25]
 [ 0.25  0.  ]
 [ 0.25  0.25]
 [ 0.25  0.5 ]
 [ 0.5  -0.5 ]
 [ 0.5  -0.25]
 [ 0.5   0.  ]
 [ 0.5   0.25]
 [ 0.5   0.5 ]]
(25, 2)


In [ ]:
# Generate the predicted input grid
Predict_Normal_Step = 1 / (Predict_Grid_Size - 1)

# Generate the input coordinate matrix
Predict_X = np.zeros((Predict_Grid_Size * Predict_Grid_Size, 2))
for i in range(0, Predict_Grid_Size * Predict_Grid_Size):
    Predict_X[i, 0] = (i // Predict_Grid_Size) * Predict_Normal_Step-0.5
    Predict_X[i, 1] = (i % Predict_Grid_Size) * Predict_Normal_Step-0.5

print(Predict_X.shape)

(10201, 2)


### Model train

In [ ]:
for Select_IMGpair in range(0, IMG_pair_num):
    # for Select_IMGpair in range(0, 1):
    print(" ")

    # Generate real samples and divide the training/prediction data
    data_y = Herr_Grid[Select_IMGpair, ::skip_step, ::skip_step].flatten()
    X_train, X_test, y_train, y_test = train_test_split(data_X, data_y, random_state=0)

    # Select models one by one
    i = 0
    for model in model_dic:
        # Train the model and test it
        param_grid = {}
        grid_search = GridSearchCV(
            model, param_grid, cv=5, n_jobs=-1, scoring="neg_root_mean_squared_error"
        )
        grid_search.fit(X_train, y_train)
        Predict_data_y = grid_search.predict(data_X)

        # Evaluate the fitting effect of the model
        model_test_1 = mean_absolute_error(data_y, Predict_data_y)
        model_test_2 = sqrt(mean_squared_error(data_y, Predict_data_y))
        model_test_3 = r2_score(data_y, Predict_data_y)
        Model_evaluation[Select_IMGpair, i] = np.array([model_test_1, model_test_2, model_test_3])
        print("Model:{}".format(model_name[i]))
        print(
            "MAE: {:.6f},RMSE: {:.6f},R2: {:.6f}".format(model_test_1, model_test_2, model_test_3)
        )

        # Predict the best positions and record the top five best positions
        Predict_y = grid_search.predict(Predict_X)
        Predict_y = np.abs(Predict_y)
        best_position_index = Predict_y.argsort()[0:5]
        for j in range(0, 5):
            Best_Initial_Position[Select_IMGpair, i, j, 0] = Predict_X[best_position_index[j], 0]
            Best_Initial_Position[Select_IMGpair, i, j, 1] = Predict_X[best_position_index[j], 1]
            Best_Initial_Position[Select_IMGpair, i, j, 2] = Predict_y[best_position_index[j]]

        print(
            "The best position:{},Y:{},Predict:{}".format(
                Best_Initial_Position[Select_IMGpair, i, :, 0],
                Best_Initial_Position[Select_IMGpair, i, :, 1],
                Best_Initial_Position[Select_IMGpair, i, :, 2],
            )
        )
        i = i + 1

### Save output

In [ ]:
# # Save prediction
# np.save("dataset/PA2/Best_Position/"+Herr_Grid_Method+str(Grid_Size)+".npy",Best_Initial_Position)

In [ ]:
# # Export the model evaluation results
# col1 = "MAE"
# col2 = "RMSE"
# col3 = "R2"

# data = pd.DataFrame(
#     {
#         col1: Model_evaluation[:, :, 0].flatten(),
#         col2: Model_evaluation[:, :, 1].flatten(),
#         col3: Model_evaluation[:, :, 2].flatten(),
#     }
# )
# data.to_excel("output/PA2/Model_Evaluation_"+Herr_Grid_Method+str(Grid_Size)+".xlsx", sheet_name="sheet1", index=False)